In [7]:
# General modules
import os
import openai
from dotenv import load_dotenv

from helper.data_preparation import load_json

# Language models
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = openai_api_key
client = openai.Client()

chat_model_name = 'gpt-4o-mini'
embed_model_name = "all-MiniLM-L6-v2"

# Paths
root_dir = r'C:\Users\fbohm\Desktop\Projects\DataScience\cluster_analysis\Data\Steamapps'
steam_title = 'Market'

path_db_analysed = os.path.join(root_dir, steam_title, "db_analysed.json")
path_db_embedded = os.path.join(root_dir, steam_title, "db_embedded.json")

path_db_clustered = os.path.join(root_dir, steam_title, "db_clustered.json")
path_db_named = os.path.join(root_dir, steam_title, "db_named.json")


# my imports
from helper.utils import *
from helper.cluster_analysis import *

configure_api(client, chat_model_name)

# Random sample

In [2]:
sample_for_embedding = os.path.join(root_dir, steam_title, "sample_for_embedding.json")

# data = read_json(path_db_analysed)
# sample_data = get_random_sample(data, 50, seed=42)
# save_to_json(sample_data, sample_for_embedding)


# Load Offline Embedding Model

In [ ]:
from sentence_transformers import SentenceTransformer
methode = "mpnet"
# paraphrase-MiniLM-L6-v2
# sentence-transformers/all-mpnet-base-v2
# Snowflake/snowflake-arctic-embed-m
# BAAI/bge-large-en-v1.5
# nomic-ai/nomic-embed-text-v1
model = SentenceTransformer('dunzhang/stella_en_1.5B_v5')

# Offline Embbeding

In [ ]:
import logging
from helper.utils import *
from sentence_transformers import SentenceTransformer

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)


def get_embedding(text, model):
    text = text.replace("\n", " ")
    embedding = model.encode(text)

    if isinstance(embedding, np.ndarray):
        embedding = embedding.tolist()

        
    return embedding


def flatten_and_embed(json_data, model, embed_key="sentence"):
    flattened_data = []
    counter = 0
    
    logger.info(f'Using {model} for embedding')
    for entry in json_data:
        # Extract common fields
        common_fields = {key: value for key, value in entry.items() if key != "topics"}
        
        if "topics" in entry and isinstance(entry["topics"], list):
            for topic in entry["topics"]:
                # Combine common fields with topic-specific fields
                flattened_entry = {**common_fields, **topic}

                # Generate embedding for the sentence
                if embed_key in topic:
                    flattened_entry["embedding"] = get_embedding(topic[embed_key], model=model)
                    counter += 1
                    if counter % 10 == 0:
                        logger.info(f"Processed {counter} entries")
                else:
                    flattened_entry["embedding"] = None
                    logger.info(f"No sentence found in entry: {entry.get('recommendationid', 'Unknown')}")
                # Append the flattened entry to the list
                flattened_data.append(flattened_entry)
        else:
            logger.warning(f"No topics found in entry: {entry.get('recommendationid', 'Unknown')}")
    
    return flattened_data


#Setup Parameters
data = read_json(sample_for_embedding)
embed_key = "sentence"

# Process the data
processed_data = flatten_and_embed(data, model, embed_key)

# Save the processed data
output_file = os.path.join(root_dir, steam_title, f"{methode}_embedding.json")

save_df_as_json(processed_data, output_file)
logger.info("Data flattening and embedding completed successfully.")

# OpenAI Embedding

In [63]:
import openai
import logging
from helper.utils import *

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)


def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   embedding = client.embeddings.create(input = [text], model=model).data[0].embedding
   return embedding


def flatten_and_embed(json_data, embed_model_name="text-embedding-3-large", embed_key="sentence"):
    """
    Flattens the topics in the JSON data and embeds the sentences.
    Args:
        json_data (list): List of JSON entries with nested topics.
        embed_model_name (str): The OpenAI embedding model name.
    Returns:
        list: A flattened list of JSON entries with embeddings.
    """
    flattened_data = []
    counter = 0
    
    for entry in json_data:
        # Extract common fields
        common_fields = {key: value for key, value in entry.items() if key != "topics"}
        
        if "topics" in entry and isinstance(entry["topics"], list):
            for topic in entry["topics"]:
                # Combine common fields with topic-specific fields
                flattened_entry = {**common_fields, **topic}
                #print(flattened_entry)
                # Generate embedding for the sentence
                if embed_key in topic:
                    flattened_entry["embedding"] = get_embedding(topic[embed_key], model=embed_model_name)
                    counter += 1
                    if counter % 10 == 0:
                        logger.info(f"Processed {counter} entries")
                else:
                    flattened_entry["embedding"] = None
                    logger.info(f"No sentence found in entry: {entry.get('recommendationid', 'Unknown')}")
                # Append the flattened entry to the list
                flattened_data.append(flattened_entry)
        else:
            logger.warning(f"No topics found in entry: {entry.get('recommendationid', 'Unknown')}")
    
    return flattened_data


data = read_json(sample_for_embedding)

# Process the data
logger.info("Flattening and embedding data...")
processed_data = flatten_and_embed(data)

# Save the processed data

output_file = os.path.join(root_dir, steam_title, "openai_embedding.json")


save_df_as_json(processed_data, output_file)

logger.info("Data flattening and embedding completed successfully.")


2024-12-16 16:26:32,670 - INFO - Flattening and embedding data...
2024-12-16 16:26:41,540 - INFO - Processed 10 entries
2024-12-16 16:26:53,474 - INFO - Processed 20 entries
2024-12-16 16:26:59,820 - INFO - Processed 30 entries


KeyboardInterrupt: 

# Matrix Evaluation

In [64]:
# Build matrix
from sklearn.model_selection import ParameterGrid
from helper.cluster_analysis import *

sample_for_embedding = os.path.join(root_dir, steam_title, "sample_for_embedding.json")
input = os.path.join(root_dir, steam_title, f"{methode}_embedding.json")

param_grid = {
    'min_cluster_size': [3, 5, 7, 10, 15],
    'min_samples': [1, 2, 3, 4, 5, 7, 10],
    'cluster_selection_epsilon': [0.1, 0.3, 0.5, 0.7, 0.9]
}
grid = ParameterGrid(param_grid)

sample_df = read_json(input)
sampled_mat = np.array([entry['embedding'] for entry in sample_df])
# Reduce dimensions

# Step 3: Reduce Dimensionality with UMAP
# Reduce the original 3075 dimensions to 20 dimensions
reducer = umap.UMAP(n_components=20, random_state=42)
mat_reduced = reducer.fit_transform(sampled_mat)

# Step 5: Apply HDBSCAN to Each Parameter Combination and Evaluate Results
results = []

for params in grid:
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=params['min_cluster_size'],
        min_samples=params['min_samples'],
        cluster_selection_epsilon=params['cluster_selection_epsilon']
    )
    labels = clusterer.fit_predict(mat_reduced)
    #print(labels)

    # Number of clusters (excluding noise, which is labeled as -1)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    # Proportion of noise points
    noise_ratio = sum(labels == -1) / len(labels)

    # Store the results for analysis
    results.append({
        'params': params,
        'n_clusters': n_clusters,
        'noise_ratio': noise_ratio
    })

# Convert results to DataFrame for easier analysis
results_df = pd.DataFrame(results)
optimal_results = results_df.sort_values(by=['noise_ratio', 'n_clusters'], ascending=[True, False])
best_params = optimal_results.iloc[0]

C:\Users\fbohm\Documents\Venvironments\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [65]:
results_df

,params,n_clusters,noise_ratio
0,"{'cluster_selection_epsilon': 0.1, 'min_cluster_size': 3, 'min_samples': 1}",34,0.092369
1,"{'cluster_selection_epsilon': 0.1, 'min_cluster_size': 3, 'min_samples': 2}",2,0.000000
2,"{'cluster_selection_epsilon': 0.1, 'min_cluster_size': 3, 'min_samples': 3}",2,0.000000
3,"{'cluster_selection_epsilon': 0.1, 'min_cluster_size': 3, 'min_samples': 4}",2,0.000000
4,"{'cluster_selection_epsilon': 0.1, 'min_cluster_size': 3, 'min_samples': 5}",2,0.000000
...,...,...,...
170,"{'cluster_selection_epsilon': 0.9, 'min_cluster_size': 15, 'min_samples': 3}",2,0.200803
171,"{'cluster_selection_epsilon': 0.9, 'min_cluster_size': 15, 'min_samples': 4}",2,0.096386
172,"{'cluster_selection_epsilon': 0.9, 'min_cluster_size': 15, 'min_samples': 5}",2,0.136546
173,"{'cluster_selection_epsilon': 0.9, 'min_cluster_size': 15, 'min_samples': 7}",2,0.204819


In [66]:
# Adjustable parameters
dimensionality_methods = ['UMAP','PCA', 'tSNE']
hdbscan_params = {"min_cluster_size": 10, "min_samples": 1, "cluster_selection_epsilon": 0.5}

# Step 3: Reduce Dimensionality with UMAP
# Reduce the original 3075 dimensions to 20 dimensions
reducer = umap.UMAP(n_components=20, random_state=42)
mat_reduced = reducer.fit_transform(sampled_mat)

open_ai_embedding = os.path.join(root_dir, steam_title, f"{methode}_embedding.json")
sample_df = load_embedded_data(open_ai_embedding)
# Apply HDBSCAN
df_total = apply_hdbscan(
    sample_df,
    mat_reduced,
    dimensionality_methods,
    hdbscan_params=hdbscan_params,
    include_2d=True,
    include_3d=True
)

# Save results
output_file = os.path.join(root_dir, steam_title, f"{methode}_2_cluster.json")
save_df_as_json(df_total, output_file)
logger.info(f"Results saved to {output_file}")

C:\Users\fbohm\Documents\Venvironments\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
2024-12-16 16:27:24,693 - INFO - Loading data from C:\Users\fbohm\Desktop\Projects\DataScience\cluster_analysis\Data\Steamapps\Market\mpnet_embedding.json
2024-12-16 16:27:24,881 - INFO - Loaded 249 valid entries with embeddings.
2024-12-16 16:27:24,884 - INFO - Applying HDBSCAN with: {'min_cluster_size': 10, 'min_samples': 1, 'cluster_selection_epsilon': 0.5}
2024-12-16 16:27:24,898 - INFO - Found 5 clusters.
2024-12-16 16:27:24,899 - INFO - Applying UMAP with 2 components.
2024-12-16 16:27:25,059 - INFO - Applying UMAP with 3 components.
2024-12-16 16:27:25,281 - INFO - Applying PCA with 2 components.
2024-12-16 16:27:25,296 - INFO - Applying PCA with 3 components.
2024-12-16 16:27:25,298 - INFO - Applying tSNE with 2 components.
2024-12-16 16:27:25,299 - INFO - Perplexity not provided, setting to 30 bas

In [67]:
from helper.cluster_naming import *
# Parameters
dimensionality_methods = ["UMAP",'PCA', "tSNE"]
clustering_algorithms = ["hdbscan"]  # No KMeans here
max_centers = 10

#kmeans_clusters = [15, 20, 25, 50]  # Number of clusters for KMeans

openai_clustered = os.path.join(root_dir, steam_title, f"{methode}_2_cluster.json")
# Load data
df_total = load_json_into_df(openai_clustered)

# Process clusters and generate names
df_total = process_clusters(
    df_total, 
    dimensionality_methods, 
    clustering_algorithms, 
    max_centers, 
    api_settings) # insert kmeans_clusters in the function when needed


# Save results
output_file = os.path.join(root_dir, steam_title, f"{methode}_3_named.json")
save_data_for_streamlit(df_total, output_file)

2024-12-16 16:27:29,259 - INFO - Loading data from C:\Users\fbohm\Desktop\Projects\DataScience\cluster_analysis\Data\Steamapps\Market\mpnet_2_cluster.json
2024-12-16 16:27:29,385 - INFO - Found 10 Topics for hdbscan_cluster_id ID: 3
2024-12-16 16:27:29,939 - INFO - Generated cluster name: Early Access Gameplay Challenges
2024-12-16 16:27:29,940 - INFO - Tokens used so far: Prompt Tokens: 7922, Completion Tokens: 223
2024-12-16 16:27:29,953 - INFO - Found 10 Topics for hdbscan_cluster_id ID: 1
2024-12-16 16:27:30,505 - INFO - Generated cluster name: Gameplay and Mechanics Overview
2024-12-16 16:27:30,506 - INFO - Tokens used so far: Prompt Tokens: 8030, Completion Tokens: 227
2024-12-16 16:27:30,511 - INFO - Found 10 Topics for hdbscan_cluster_id ID: 0
2024-12-16 16:27:31,111 - INFO - Generated cluster name: Game Enthusiasm and Future Outlook
2024-12-16 16:27:31,112 - INFO - Tokens used so far: Prompt Tokens: 8230, Completion Tokens: 233
2024-12-16 16:27:31,116 - INFO - Found 10 Topics 